In [1]:
import pandas as pd
import pyodbc

# -------------------------
# SQL Server connection
# -------------------------
server = '157.119.230.120,4071'
database = 'PowerBI_LISV2'
username = 'sa'
password = 'Y0m@Sql.!.123'

conn_str = f"""
    DRIVER={{ODBC Driver 17 for SQL Server}};
    SERVER={server};
    DATABASE={database};
    UID={username};
    PWD={password};
    TrustServerCertificate=yes;
"""

conn = pyodbc.connect(conn_str)

# -------------------------
# Client list
# -------------------------
x = [
    'AmulManagement', 'CholayilManagement'
]

# -------------------------
# Store summary info
# -------------------------
summary_data = []

# -------------------------
# Fetch data per client
# -------------------------
for client in x:
    query = f"""
        SELECT *,
                VisitId AS UserKey
        FROM {client}.AdhocVisibilityReportExport
        WHERE Month = '1' AND Year = '2026';
    """

    df = pd.read_sql(query, conn)

    # Clean column names
    df.columns = df.columns.str.strip()

    
    # Create dynamic DataFrame variable
    df_name = f"{client}_Free_Visibility"
    globals()[df_name] = df

    print(f"{df_name} created with {len(df)} rows")

    # Collect summary info
    summary_data.append({
        "ClientName": client,
        "No_of_Rows": len(df)
    })

# -------------------------
# Client-wise summary (ONLY ONCE)
# -------------------------
summary_df = pd.DataFrame(summary_data)
print("\nClient-wise Free Visibility Summary:")
print(summary_df)


C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_17724\4055630987.py:46: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


AmulManagement_Free_Visibility created with 2744 rows
CholayilManagement_Free_Visibility created with 1067 rows

Client-wise Free Visibility Summary:
           ClientName  No_of_Rows
0      AmulManagement        2744
1  CholayilManagement        1067


C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_17724\4055630987.py:46: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [2]:
import pandas as pd
Amul_Free_Visibility = pd.read_csv(r"C:\Users\Administrator\LIS\Rahul\Free_Visibility\Amul_FreeVisibility.csv")

# Create new column: UserName-Day
Amul_Free_Visibility["UserName_Day"] = Amul_Free_Visibility["VisitId"]

Cholayil_Free_Visibility = pd.read_csv(r"C:\Users\Administrator\LIS\Rahul\Free_Visibility\Cholayil_FreeVisibility.csv")

# Create new column: UserName-Day
Cholayil_Free_Visibility["UserName_Day"] = Cholayil_Free_Visibility["VisitId"]



In [3]:
# Amul = pd.merge(
#     Amul_Store_Attendance,
#     AmulManagement_Store_Attendance, 
#     left_on="UserName_Day",
#     right_on="UserKey",
#     how="left"
# )

# print("Number of unmatched rows for Amul:", Amul['UserName_Day'].isna().sum())

# Cholayil = pd.merge(
#     Cholayil_Store_Attendance,
#     CholayilManagement_Store_Attendance,
#     left_on="UserName_Day",
#     right_on="UserKey",
#     how="left"
# )

# print("Number of unmatched rows for Cholayil:", Cholayil['UserName_Day'].isna().sum())


In [4]:
import pandas as pd
from datetime import datetime
import os

# -------------------------
# Current date
# -------------------------
today_str = datetime.now().strftime("%Y-%m-%d")

# -------------------------
# Client list
# -------------------------
clients = [
    'Amul', 'AdaniWilmer', 'Cholayil', 'DanoneGT', 'DanoneMT',
    'GCPLFoods', 'GCPL', 'JohnsonJohnson', 'Parle',
    'Colgate', 'Pidilite', 'Piramal', 'ZydusMT', 'ZydusGT'
]

# -------------------------
# Store Attendance summary
# -------------------------
unmatched_summary = []

# -------------------------
# Merge & calculate unmatched rows
# -------------------------
for client in clients:

    left_df = globals().get(f"{client}_Free_Visibility")
    right_df = globals().get(f"{client}Management_Free_Visibility")

    if left_df is None or right_df is None:
        print(f"Skipping {client}: DataFrame missing")
        continue

    merged_df = pd.merge(
        left_df,
        right_df,
        left_on="UserName_Day",
        right_on="UserKey",
        how="left"
    )

    unmatched_count = merged_df['UserKey'].isna().sum()

    unmatched_summary.append({
        "Date": today_str,
        "ClientName": client,
        "Portal_Records": len(left_df),
        "SQL_Records": len(right_df),
        "Unmatched_Rows": unmatched_count
    })

    globals()[f"{client}_Merged"] = merged_df

    print(f"{client} → Unmatched Rows: {unmatched_count}")

# -------------------------
# Final Summary DataFrame
# -------------------------
unmatched_summary_df = pd.DataFrame(unmatched_summary)

print("\nClient-wise Unmatched Summary:")
print(unmatched_summary_df)

# -------------------------
# CSV Path
# -------------------------
output_path = "FreeVisibility_Unmatched_Summary.csv"

# -------------------------
# Append if file exists, else create new
# -------------------------
if os.path.exists(output_path):
    unmatched_summary_df.to_csv(output_path, mode='a', header=False, index=False)
else:
    unmatched_summary_df.to_csv(output_path, index=False)

print(f"\nSummary CSV updated successfully at: {output_path}")


Amul → Unmatched Rows: 0
Skipping AdaniWilmer: DataFrame missing
Cholayil → Unmatched Rows: 0
Skipping DanoneGT: DataFrame missing
Skipping DanoneMT: DataFrame missing
Skipping GCPLFoods: DataFrame missing
Skipping GCPL: DataFrame missing
Skipping JohnsonJohnson: DataFrame missing
Skipping Parle: DataFrame missing
Skipping Colgate: DataFrame missing
Skipping Pidilite: DataFrame missing
Skipping Piramal: DataFrame missing
Skipping ZydusMT: DataFrame missing
Skipping ZydusGT: DataFrame missing

Client-wise Unmatched Summary:
         Date ClientName  Portal_Records  SQL_Records  Unmatched_Rows
0  2026-01-07       Amul            2744         2744               0
1  2026-01-07   Cholayil            1067         1067               0

Summary CSV updated successfully at: FreeVisibility_Unmatched_Summary.csv
